# Claude CLI - Automatisation Avancee

**Module :** Vibe-Coding / Claude Code / Notebooks CLI  
**Niveau :** Avance  
**Duree :** 30 min  
**Prerequis :** Notebooks 01-04 completes

## Objectifs d'Apprentissage

- [ ] Creer des pipelines d'automatisation avec subprocess
- [ ] Integrer Claude dans des scripts Bash/PowerShell
- [ ] Comprendre les slash commands (/commit, /review)
- [ ] Decouvrir les hooks (PreToolUse, PostToolUse)
- [ ] Construire un script de revue de code automatise

---

> **Pourquoi automatiser avec Claude CLI ?**
> - **Repetabilite** : Memes analyses appliquees systematiquement a tout un projet
> - **Integration CI/CD** : Revues de code automatiques avant merge
> - **Scalabilite** : Traitement de centaines de fichiers sans intervention manuelle
> - **Coherence** : Memes criteres appliques a chaque revue

**Architecture d'automatisation typique :**
```
Declencheur (git hook, CI, cron)
        |
        v
    Script Python/Bash
        |
        v
    Claude CLI (subprocess)
        |
        v
    Sortie structuree (JSON/Markdown)
        |
        v
    Action (commit, notification, rapport)
```

## 1. Configuration

In [ ]:
import sys
import subprocess
import json
import os
from pathlib import Path

sys.path.insert(0, 'helpers')
from claude_cli import run_claude, verify_installation, print_response

EXAMPLES_DIR = Path('examples/sample_project')
print(f"Claude CLI pret: {verify_installation()}")

## 2. Pipelines avec Subprocess

Les pipelines permettent de chainer des operations Claude pour des workflows complexes :

```
Input → [Claude: Analyse] → [Claude: Correction] → [Claude: Review] → Output
```

### Avantages des pipelines

- **Decomposition** : Chaque etape est specialisee et testable independamment
- **Flexibilite** : Facile d'ajouter/supprimer des etapes
- **Tracabilite** : Resultats intermediaires conserves pour debug

### Considerations importantes

> **Attention au cout API**  
> Chaque etape = un appel API. Un pipeline de 3 etapes sur 10 fichiers = 30 appels.
> Utilisez `haiku` pour les etapes intermediaires et `sonnet` pour l'etape finale si qualite requise.

In [ ]:
def create_pipeline(*steps):
    """
    Cree un pipeline de prompts Claude.
    
    Args:
        steps: Liste de tuples (nom, prompt_template)
        
    Returns:
        Fonction qui execute le pipeline.
    """
    def execute(initial_input):
        current_data = initial_input
        results = []
        
        for step_name, prompt_template in steps:
            prompt = prompt_template.format(input=current_data)
            stdout, stderr, code = run_claude(prompt, model="haiku")
            
            if code != 0:
                return {"error": f"Etape '{step_name}' echouee", "stderr": stderr}
            
            results.append({"step": step_name, "output": stdout})
            current_data = stdout
        
        return {"success": True, "results": results, "final_output": current_data}
    
    return execute

print("Fonction create_pipeline definie")

In [ ]:
# Exemple de pipeline : Analyse → Resume → Ameliorations
code_review_pipeline = create_pipeline(
    ("analyse", "Analyse ce code et liste ses caracteristiques principales:\n{input}"),
    ("resume", "Resume cette analyse en 3 points cles:\n{input}"),
    ("ameliorations", "Propose 2 ameliorations basees sur ce resume:\n{input}")
)

print("Pipeline de revue de code defini")

In [ ]:
# Test du pipeline (ATTENTION: 3 appels API)
sample_code = '''
def process_data(items):
    result = []
    for item in items:
        if item > 0:
            result.append(item * 2)
    return result
'''

# Decommentez pour executer
# output = code_review_pipeline(sample_code)
# if output.get("success"):
#     for r in output["results"]:
#         print(f"\n=== {r['step'].upper()} ===")
#         print(r['output'][:300])

print("(Pipeline non execute - decommentez pour tester)")

## 3. Integration Scripts Shell

Claude Code peut etre integre dans des scripts Bash ou PowerShell pour des workflows systeme.

> **Note syntaxe** : Les `$` dans les exemples ci-dessous sont des variables shell standard (Bash/PowerShell), pas du formatage special.

### Exemple Bash (Linux/macOS/WSL)

```bash
#!/bin/bash
# Script de revue automatique

FILE=$1                                          # Premier argument = fichier
REVIEW=$(claude -p "Revue de code pour: $(cat $FILE)")  # Execution Claude
echo "$REVIEW" > "${FILE}.review.md"             # Sauvegarde resultat
```

### Exemple PowerShell (Windows)

```powershell
# Analyse de tous les fichiers Python du repertoire courant
Get-ChildItem -Filter *.py | ForEach-Object {
    $content = Get-Content $_.FullName -Raw      # Lecture fichier entier
    $review = claude -p "Resume ce fichier: $content"
    $review | Out-File "$($_.BaseName).review.md"
}
```

### Bonnes pratiques scripts

| Pratique | Raison |
|----------|--------|
| Toujours verifier le code de retour | Detecter les echecs Claude |
| Utiliser des quotes doubles | Preserver les espaces dans le code |
| Limiter la taille du contenu | Eviter les timeouts sur gros fichiers |
| Logger les appels | Debug et audit |

In [ ]:
# Generer un script de revue automatique
bash_script = '''#!/bin/bash
# Script de revue de code automatique avec Claude

FILE="$1"

if [ -z "$FILE" ]; then
    echo "Usage: $0 <fichier.py>"
    exit 1
fi

if [ ! -f "$FILE" ]; then
    echo "Fichier non trouve: $FILE"
    exit 1
fi

echo "Analyse de $FILE..."
CONTENT=$(cat "$FILE")

REVIEW=$(claude -p "Effectue une revue de code pour ce fichier Python. 
Identifie: bugs potentiels, problemes de style, ameliorations possibles.

Code:
$CONTENT")

OUTPUT="${FILE%.py}.review.md"
echo "# Code Review: $FILE" > "$OUTPUT"
echo "" >> "$OUTPUT"
echo "$REVIEW" >> "$OUTPUT"

echo "Revue sauvegardee dans: $OUTPUT"
'''

print(bash_script)

In [ ]:
# Version Python equivalente
def auto_review(filepath):
    """Effectue une revue de code automatique."""
    path = Path(filepath)
    
    if not path.exists():
        return {"error": f"Fichier non trouve: {filepath}"}
    
    content = path.read_text()
    
    stdout, stderr, code = run_claude(
        f"""Effectue une revue de code pour ce fichier Python.
Identifie:
1. Bugs potentiels
2. Problemes de style (PEP 8)
3. Ameliorations possibles
4. Securite

Format: sections avec titres markdown.

Code:
{content}""",
        model="sonnet"
    )
    
    if code == 0:
        output_path = path.with_suffix('.review.md')
        output_path.write_text(f"# Code Review: {path.name}\n\n{stdout}")
        return {"success": True, "review": stdout, "output_file": str(output_path)}
    else:
        return {"error": stderr}

print("Fonction auto_review definie")

## 4. Slash Commands

Claude Code propose des commandes integrees accessibles avec `/` en mode interactif :

| Commande | Description | Exemple d'utilisation |
|----------|-------------|----------------------|
| `/init` | Genere un CLAUDE.md pour le projet | Debut de projet |
| `/commit` | Cree un commit avec message genere | Apres modifications |
| `/review` | Revue des changements en cours | Avant commit |
| `/status` | Statut de connexion et quota | Debug connexion |
| `/mcp` | Statut des serveurs MCP | Verifier outils disponibles |
| `/help` | Liste toutes les commandes | Decouverte |

> **Mode interactif uniquement** : Ces commandes fonctionnent dans `claude` (sans `-p`), pas dans `claude -p "..."`.

### Exemple de workflow typique

```bash
# En mode interactif
$ claude
> /review            # Voir les changements non commites
> /commit            # Generer message et commiter
> /status            # Verifier que tout est OK
```

Ces commandes sont utilisees en mode interactif (`claude` sans `-p`).

In [ ]:
# Simuler /commit : generer un message de commit
def generate_commit_message(diff_content):
    """Genere un message de commit a partir d'un diff."""
    stdout, stderr, code = run_claude(
        f"""Genere un message de commit conventionnel pour ces changements.

Format attendu:
type(scope): description courte

Description detaillee si necessaire.

Types: feat, fix, docs, style, refactor, test, chore

Diff:
{diff_content}""",
        model="haiku"
    )
    return stdout if code == 0 else f"Erreur: {stderr}"

# Exemple de diff
example_diff = '''
--- a/utils.py
+++ b/utils.py
@@ -15,6 +15,10 @@ def calculate_statistics(data):
+    if not data:
+        raise ValueError("Empty data")
+
     mean = sum(data) / len(data)
'''

# Decommentez pour tester
# message = generate_commit_message(example_diff)
# print(message)

print("Fonction generate_commit_message definie")

## 5. Hooks (Concept)

Les **hooks** permettent d'intercepter les actions de Claude pour les modifier ou les valider avant/apres execution.

### Types de hooks

| Type | Declenchement | Cas d'usage |
|------|---------------|-------------|
| **PreToolUse** | Avant l'utilisation d'un outil | Validation, confirmation, audit |
| **PostToolUse** | Apres l'utilisation | Logging, notification, cleanup |
| **Notification** | Sur certains evenements | Alertes, metriques |

### Configuration

Les hooks se configurent dans `.claude/settings.json` au niveau du projet :

```json
{
  "hooks": {
    "PreToolUse": {
      "Write": "python validate_write.py",
      "Bash": "python confirm_bash.py"
    },
    "PostToolUse": {
      "Bash": "python log_bash.py"
    }
  }
}
```

### Protocole de communication

Le hook recoit les parametres en JSON sur stdin et doit retourner :
- **Code 0** : Action approuvee
- **Code non-0** : Action refusee (avec message sur stderr)

### Cas d'usage courants

- **Securite** : Bloquer l'ecriture de secrets dans le code
- **Audit** : Logger toutes les commandes Bash executees
- **Validation** : Verifier le format des fichiers avant ecriture

In [ ]:
# Exemple de hook de validation
def validate_write_hook(filepath, content):
    """Hook qui valide le contenu avant ecriture."""
    issues = []
    
    # Verifications de securite
    if 'password' in content.lower() and '=' in content:
        issues.append("Possible mot de passe en clair detecte")
    
    if 'api_key' in content.lower():
        issues.append("Possible cle API detectee")
    
    # Verifications de style
    if filepath.endswith('.py'):
        if 'import *' in content:
            issues.append("Import wildcard detecte (mauvaise pratique)")
    
    return {
        "approved": len(issues) == 0,
        "issues": issues
    }

# Test
test_content = '''
from utils import *
api_key = "sk-1234"
'''

result = validate_write_hook("config.py", test_content)
print(f"Approuve: {result['approved']}")
print(f"Problemes: {result['issues']}")

## 6. Script de Revue Automatise Complet

Cette section presente un exemple complet et reutilisable de revue de code automatisee.

### Architecture du CodeReviewer

```
CodeReviewer
    |
    +-- review_file(path)      # Revue d'un fichier unique
    |       |
    |       +-- check: bugs
    |       +-- check: style  
    |       +-- check: security
    |       +-- check: performance
    |
    +-- review_project(dir)    # Revue de tout un projet
    |
    +-- generate_report()      # Generation rapport Markdown
```

> **Extensibilite** : La classe est concue pour etre etendue. Ajoutez vos propres checks dans l'exercice 3.

In [ ]:
class CodeReviewer:
    """Revue de code automatisee avec Claude."""
    
    def __init__(self, model="sonnet"):
        self.model = model
        self.checks = [
            ("bugs", "Identifie les bugs potentiels"),
            ("style", "Verifie le style PEP 8"),
            ("security", "Analyse les problemes de securite"),
            ("performance", "Identifie les optimisations possibles"),
        ]
    
    def review_file(self, filepath):
        """Effectue une revue complete d'un fichier."""
        path = Path(filepath)
        if not path.exists():
            return {"error": "Fichier non trouve"}
        
        content = path.read_text()
        results = {"file": str(path), "checks": {}}
        
        for check_name, check_desc in self.checks:
            stdout, stderr, code = run_claude(
                f"{check_desc} dans ce code (reponse courte):\n\n{content}",
                model="haiku"  # Rapide pour chaque check
            )
            results["checks"][check_name] = stdout if code == 0 else f"Erreur: {stderr}"
        
        return results
    
    def review_project(self, directory, pattern="*.py"):
        """Revue tous les fichiers d'un projet."""
        path = Path(directory)
        files = list(path.rglob(pattern))
        
        return {
            "directory": str(path),
            "files_count": len(files),
            "files": [str(f) for f in files]
            # En production, on appellerait review_file pour chaque fichier
        }
    
    def generate_report(self, results):
        """Genere un rapport markdown."""
        lines = [f"# Code Review: {results['file']}\n"]
        
        for check, output in results.get("checks", {}).items():
            lines.append(f"## {check.title()}\n")
            lines.append(output + "\n")
        
        return "\n".join(lines)

print("CodeReviewer defini")

In [ ]:
# Test du reviewer (ATTENTION: multiple appels API)
reviewer = CodeReviewer()

# Voir les fichiers disponibles
project_info = reviewer.review_project(EXAMPLES_DIR)
print(f"Fichiers dans le projet: {project_info['files_count']}")
for f in project_info['files']:
    print(f"  - {f}")

In [ ]:
# Revue d'un fichier (decommentez pour executer - 4 appels API)
# results = reviewer.review_file(EXAMPLES_DIR / "utils.py")
# report = reviewer.generate_report(results)
# print(report)

print("(Revue non executee - decommentez pour tester)")

## 7. Exercices

In [ ]:
# EXERCICE 1 : Pipeline de generation de docstrings
#
# Objectif : Creer un pipeline qui:
# 1. Lit un fichier Python
# 2. Identifie les fonctions sans docstring
# 3. Genere les docstrings manquantes
#
# Indices :
# - Utilisez create_pipeline() defini plus haut
# - Etape 1 : "Liste les fonctions sans docstring dans ce code"
# - Etape 2 : "Pour chaque fonction, genere une docstring Google-style"
# - Testez avec examples/sample_project/utils.py

# Votre code ici
# docstring_pipeline = create_pipeline(
#     ("identify", "Liste les fonctions sans docstring:\n{input}"),
#     ("generate", "Genere des docstrings Google-style pour:\n{input}")
# )

print("Decommentez et completez le code pour executer l'exercice")

In [ ]:
# EXERCICE 2 : Hook de verification des tests
#
# Objectif : Creer un hook qui verifie que tout fichier Python
# a au moins un fichier de test associe.
#
# Regles :
# - fichier.py doit avoir test_fichier.py ou fichier_test.py
# - Les fichiers __init__.py sont exemptes
# - Les fichiers dans un dossier tests/ sont exemptes
#
# Signature :
# def check_test_exists(filepath: str) -> dict:
#     """Retourne {"approved": bool, "reason": str}"""

# Votre code ici
# from pathlib import Path
# def check_test_exists(filepath):
#     path = Path(filepath)
#     if path.name == "__init__.py":
#         return {"approved": True, "reason": "init file"}
#     # Completez...

print("Decommentez et completez le code pour executer l'exercice")

In [ ]:
# EXERCICE 3 : Extension de CodeReviewer
#
# Objectif : Ajouter un check supplementaire a CodeReviewer.
#
# Suggestions de checks :
# - "documentation" : Verifie que les fonctions publiques ont des docstrings
# - "complexity" : Identifie les fonctions trop longues (>50 lignes)
# - "typing" : Verifie la presence de type hints
# - "imports" : Detecte les imports inutilises
#
# Structure :
# class ExtendedReviewer(CodeReviewer):
#     def __init__(self):
#         super().__init__()
#         self.checks.append(("votre_check", "Description du check"))

# Votre code ici

print("Decommentez et completez le code pour executer l'exercice")

## 8. Resume Final

### Competences acquises dans cette serie

| Notebook | Competences |
|----------|-------------|
| **01-Bases** | `claude -p`, modeles, formats de sortie |
| **02-Sessions** | `-c`, sessions, conversations multi-tours |
| **03-References** | @fichier, plages de lignes, CLAUDE.md |
| **04-Agents** | Explore, Plan, subagents, parallelisation |
| **05-Automatisation** | Pipelines, scripts shell, hooks, revue automatisee |

### Points cles de ce notebook

- Les **pipelines** permettent de chainer des operations Claude de maniere modulaire
- L'integration **shell** (Bash/PowerShell) permet d'automatiser au niveau systeme
- Les **slash commands** accelerent les taches courantes en mode interactif
- Les **hooks** offrent un controle fin sur les actions de Claude

### Documentation et ressources

- **Documentation officielle** : [docs.anthropic.com/claude-code](https://docs.anthropic.com/en/docs/claude-code)
- **GitHub Claude Code** : [github.com/anthropics/claude-code](https://github.com/anthropics/claude-code)
- **Concepts avances** : [CONCEPTS-AVANCES.md](../../docs/claude-code/CONCEPTS-AVANCES.md)
- **Modeles alternatifs** : [OPENROUTER-MODELES-ALTERNATIFS.md](../../docs/claude-code/OPENROUTER-MODELES-ALTERNATIFS.md)

### Ressources communautaires

- [Awesome Claude Code](https://github.com/hesreallyhim/awesome-claude-code)
- [SkillsMP Marketplace](https://skillsmp.com/)

---

**Felicitations !** Vous avez complete la serie de notebooks Claude CLI.

> **Prochaines etapes suggeres**  
> - Explorez les notebooks GenAI pour la generation d'images
> - Creez votre propre CLAUDE.md pour vos projets
> - Configurez des hooks de securite pour votre equipe